# Launch Grid Network on Heroku
In this tutorial, you'll learn how to deploy a grid network on Heroku platform and then interact with it using PySyft.

_WARNING: Grid nodes publish datasets online and are for EXPERIMENTAL use only. Deploy nodes at your own risk. Do not use OpenGrid with any data/models you wish to keep private._

In order to run an grid network remotely you will need to run two different apps: a grid gateway and one or more grid workers. In this tutorial we will use the websocket app available [here](https://github.com/OpenMined/Grid/tree/dev/app/websocket) to start the grid workers.


### Step 1: Download the repository

```bash
git clone https://github.com/OpenMined/PyGrid/
```


### Step 2: Make grid importable

Install grid as a python package

```bash
cd PyGrid
python setup.py install (or python setup.py develop)
```

### Step 3: Use Grid Library to Perform Deployment

#### Import dependencies

In [3]:
import grid as gr
import syft as sy
import torch as th
import torch.nn.functional as F


from grid.deploy import HerokuGatewayDeployment, HerokuNodeDeployment
hook = sy.TorchHook(th)

#### Setup gateway's environment deploy configs

In [4]:
gateway_url_field = "opengridgateway"
# Set environment variables
environment = { "NUM_REPLICAS" : "1",
                "SECRET_KEY" : "secretneededhere" }

gateway_component = HerokuGatewayDeployment(gateway_url_field,env_vars=environment,branch="add_gateway_persistence")

#### Perform gateway deployment

In [4]:
gateway_component.deploy()

Step 0: Checking Dependencies
	Checking for git dependency...DONE!
	Checking for heroku --version dependency...DONE!
	Checking for pip dependency...DONE!
	Checking to see if heroku is logged in...DONE!

Step 1: Making sure app name 'opengridgateway' is available
	['https://opengridgateway.herokuapp.com/ | https://git.heroku.com/opengridgateway.git\n']

Step 3: Cleaning up heroku check ...

Step 4: cleaning up git...
	
Step 5: cloning heroku app code from Github...
	
Step 6: copying app code from cloned repo...
	
Step 7: removing the rest of the cloned code...
	
Step 8: Initializing new github (for Heroku)...
	Initialized empty Git repository in /home/ionesio/workspace/notebooks/PyGrid/examples/tmp/.git/
	
Step 9: Adding files to heroku github...
	
Step 10: Committing files to heroku github...
	[master (root-commit) 7b9f7bd] init
	 10 files changed, 438 insertions(+)
	 create mode 100644 Dockerfile
	 create mode 100644 Procfile
	 create mode 100644 app/__init__.py
	 create mode 100644 a

'https://opengridgateway.herokuapp.com'

#### Perform nodes deployment

In [11]:
ids = { "aicompanynode", "labresearchnode", "hospitalnode"}

for node_id in ids:
    environment = {
        "ID": node_id,
        "GRID_NETWORK_URL": "http://{}.herokuapp.com".format(gateway_url_field),
        "ADDRESS": "http://{}.herokuapp.com/".format(node_id)
    }
    node_component = HerokuNodeDeployment(node_id, env_vars=environment)
    node_component.deploy()

Step 0: Checking Dependencies
	Checking for git dependency...DONE!
	Checking for heroku --version dependency...DONE!
	Checking for pip dependency...DONE!
	Checking to see if heroku is logged in...DONE!

Step 1: Making sure app name 'labresearchnode' is available
	['https://labresearchnode.herokuapp.com/ | https://git.heroku.com/labresearchnode.git\n']

Step 3: Cleaning up heroku/postgres checks...

Step 4: cleaning up git...
	
Step 5: cloning heroku app code from Github...
	
Step 6: copying app code from cloned repo...
	
Step 7: removing the rest of the cloned code...
	
Step 8: Initializing new github (for Heroku)...
	Initialized empty Git repository in /home/ionesio/workspace/notebooks/PyGrid/examples/tmp/.git/
	
Step 9: Adding files to heroku github...
	
Step 10: Committing files to heroku github...
	[master (root-commit) 7439262] init
	 15 files changed, 1357 insertions(+)
	 create mode 100644 Dockerfile
	 create mode 100644 LICENSE
	 create mode 100644 Procfile
	 create mode 100644

### Step 6: Start communication with the Grid Gateway and workers

Let's start communication with the Gateway and the workers.

In [5]:
gateway_url = "http://{}.herokuapp.com".format(gateway_url_field)

my_grid = gr.GridNetwork(gateway_url)

In [13]:
# Build Model
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = th.nn.Linear(2, 1)
        self.bias = th.tensor([1000.0])
        self.state += ["fc1", "bias"]

    def forward(self, x):
        x = self.fc1(x)
        return F.log_softmax(x, dim=0) + self.bias

model = Net()
model.build(th.tensor([1.0, 2]))


In [14]:
my_grid.serve_model(model, model_id="simple_model",allow_remote_inference=True, allow_download=False)

In [19]:
my_grid.run_remote_inference(model_id="simple_model", data=th.tensor([1.0, 2]))

tensor([1000.])

In [6]:
worker = my_grid.query_model(model_id="simple_model")
print("This model was hosted at: ", worker.id)


This model was hosted at:  aicompanynode
